# WB RecSys Project

# Общее описание проекта

Необходимо на основании взаимодействий пользователей с товарами предсказать следующие взаимодействия пользователей с товарами.

# Stage 3

- Сформировать обучающую выборку
- Спроектировать схему валидации с учетом специфики задачи
- Обосновать выбор способа валидации


# Preprocessing images

# Импорт библиотек

# Данные

In [1]:
import os
import numpy as np
import polars as pl
import dill
import tqdm

from sklearn.decomposition import PCA

from PIL import Image

import torch
from torch.utils.data import Dataset

from transformers import CLIPProcessor, CLIPModel

Путь до данных

In [2]:
data_path = "../../data/closed/"
data_load_path = "../../data/load/"
img_data_path = data_load_path + "images/"

## Images data

### Чтение данных

Посмотрим для каких айтемов отсутствуют картинки

In [3]:
# Sorded list with existing items
item_id_list = (
    pl.scan_parquet(data_path + "df_items.parquet")
    .select("item_id")
    .sort(by="item_id")
    .collect()
    .to_numpy()
    .flatten()
    .tolist()
)

In [5]:
items_without_img = []

for i in item_id_list:
    if not (os.path.exists(img_data_path + f"{i}.jpg")):
        items_without_img.append(i)

In [ ]:
len(items_without_img)

Всего таких айтемов 269

In [ ]:
os.path.join(img_data_path, f"{1}.jpg")

## CLIP: Zero-shot

In [8]:
# Получим отсортированный список лейблов изображений,
# для которых присутствуют картиики
labels = sorted(list(set(item_id_list) - set(items_without_img)))

images_paths = [
    img_data_path + f"{idx}.jpg" for idx in labels
]

In [9]:
# Размер изображения, к которому будем приводить
IMG_SIZE = 128

In [10]:
# Зададим свой датасет
class CustomImageDataset(Dataset):
    def __init__(
        self,
        file_paths,
        labels,
        transform=None,
        image_size: int= 128,
    ):
        self.file_paths = file_paths
        self.labels = labels
        self.transform = transform
        self.image_size = image_size

    def __len__(self):
        return len(self.file_paths)

    def __getitem__(self, index):
        image = Image.open(self.file_paths[index]).resize(
            size=(self.image_size, self.image_size),
        )
        label = self.labels[index]

        if self.transform:
            image = self.transform(image)

        return image, label

In [11]:
# Инициализируем датасет
imgs_dataset = CustomImageDataset(
    images_paths, #[:128  * 20],
    labels=labels, #[:128  * 20],
    image_size=IMG_SIZE,
)

### Image embeddings

Extract image embeddings on both subsets using pretrained CLIP.

In [12]:
# Set device
device = "cuda" if torch.cuda.is_available() else "cpu"

# Load model
model = CLIPModel.from_pretrained(
    "openai/clip-vit-base-patch32",
).to(device)
processor = CLIPProcessor.from_pretrained(
    "openai/clip-vit-base-patch32",
)

model.eval();

In [13]:
def imgs_collate_fn(batch):
    images, labels = zip(*batch)
    images = [
        processor(
            images=image,
            return_tensors="pt",
        )["pixel_values"]
        for image in images
    ]
    images = torch.cat(images, dim=0)
    labels = torch.tensor(labels)
    return images, labels


batch_size = 512

# Get dataloader
imgs_loader = torch.utils.data.DataLoader(
    imgs_dataset,
    batch_size=batch_size,
    shuffle=False,
    collate_fn=imgs_collate_fn,
)

In [14]:
def extract_embeddings(
    model, loader, device=device
) -> tuple[torch.Tensor, torch.Tensor]:
    all_embeddings = []
    all_labels = []

    with torch.no_grad():  # Отключаем градиенты для ускорения
        for images, labels in tqdm.tqdm(iter(loader)):
            # Перемещаем данные на указанное устройство
            images = images.to(device)

            # Извлекаем эмбеддинги из модели
            embeddings = model.get_image_features(images)

            # Сохраняем результаты
            all_embeddings.append(embeddings)
            all_labels.append(labels)

    # Объединяем все батчи в один тензор
    all_embeddings = torch.cat(all_embeddings, dim=0).to("cpu")
    all_labels = torch.cat(all_labels, dim=0)

    return all_embeddings, all_labels

In [ ]:
img_embeds, img_labels = extract_embeddings(model, imgs_loader)

In [ ]:
img_labels, img_labels.shape

In [ ]:
img_embeds, img_embeds.shape

In [22]:
# Сохраним в бинарник эмбединги катинок
with open(data_path + "img_embeds.dill", "wb") as f:
    dill.dump(img_embeds, f)

# Сохраним в бинарник лейблы картинок
with open(data_path + "img_labels.dill", "wb") as f:
    dill.dump(img_labels, f)

## Снизим размерность

In [ ]:
# Загрузим данные
with open(data_path + "img_embeds.dill", "rb") as f:
    img_embeds = dill.load(f)

# Загрузим данные
with open(data_path + "img_labels.dill", "rb") as f:
    img_labels = dill.load(f)

In [ ]:
components_to_keep = 10

pca_lowrank = PCA(n_components=components_to_keep)
all_embeddings = pca_lowrank.fit_transform(img_embeds)
all_embeddings.shape
img_embeds_df = pl.concat(
    [
        pl.DataFrame(
            data=img_labels.numpy(),
            schema=["item_id"],
        ),
        pl.DataFrame(
            data=all_embeddings,
            schema=[f"img_emb_pca_{i}" for i in range(components_to_keep)],
        ),
    ],
    how="horizontal",
)

img_embeds_df

## Сольем данные в одну таблицу

In [22]:
(
    pl.scan_parquet(data_path + "df_items.parquet")
    .collect()
    .join(other=img_embeds_df, on="item_id", how="left")
    # Заполняем пропуски товаров, для которых нет изображений
    .fill_nan(0)
    .fill_null(0)
).write_parquet(data_path + "df_items.parquet")

In [ ]:
pl.scan_parquet(data_path + "df_items.parquet").schema